# Cloudbutton Geospatial: Water Consumption Workflow

---

In [1]:
import sys
sys.path.append('../')

In [109]:
from collections import defaultdict
from cloudbutton_geospatial.io_utils.plot import plot_results
from cloudbutton_geospatial.utils.notebook import date_picker
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from shapely.geometry import Point, MultiPoint, box
from pprint import pprint
import functools
import datetime
import os
import shutil
import math
import numpy as np
import pandas as pd
import lithops
import requests
import rasterio
import json
import random
import re
import tempfile
import concurrent.futures
from IPython.display import Image
import matplotlib.pyplot as plt
from lithops.storage import Storage
from lithops.storage.utils import StorageNoSuchKeyError

## Workflow parameters

Area outside the processed tile that we want to consider for taking SIAM stations into account:

In [3]:
AREA_OF_INFLUENCE = 4000

Lithops Variables:

In [27]:
DATA_BUCKET = 'geospatial-usecase'
COMPUTE_BACKEND = 'localhost'
STORAGE_BACKEND = 'localhost'
RUNTIME = None

In [89]:
DTM_ASC_PREFIX = 'DTMs/asc/'
DTM_GEOTIFF_PREFIX = 'DTMs/geotiff/'

Split tile into square chunks (number of tiles = SPLITS^2):

In [5]:
SPLITS = 2

Correlation coefficient between elevation and temperature:

In [6]:
r = -0.0056

Elevation to interpolate temperature:

In [7]:
zdet = 2000

Day of year to calculate solar irradiation:

In [8]:
date = date_picker(default=datetime.date(2022, 5, 15))

DatePicker(value=datetime.date(2022, 5, 15), description='Pick a Date')

In [9]:
DAY_OF_YEAR = date.value.timetuple().tm_yday
DAY_OF_YEAR

135

## Data preparation

In [25]:
storage = lithops.storage.Storage(backend=STORAGE_BACKEND)

Get meteorological data for selected date (12:00h) and station locations from Catalonia Open Data portal

In [37]:
XEMA_DATA_URL = 'https://analisi.transparenciacatalunya.cat/resource/nzvn-apee.json'
XEMA_META_STATIONS = 'https://analisi.transparenciacatalunya.cat/resource/yqwd-vj5e.json'
XEMA_META_VARS = 'https://analisi.transparenciacatalunya.cat/resource/4fb2-n3yi.json'

@functools.cache
def get_meteo_stations():
    if os.path.exists('meta_cache/meteo_stations.csv'):
        print('Getting meteo station from local cache')
        with open('meta_cache/meteo_stations.csv', 'r') as f:
            stations_meta = pd.read_csv(f)
        return stations_meta
    
    print('Fetching meteo station from analisi.transparenciacatalunya.cat...')
    res = requests.get(XEMA_META_STATIONS, params={'NOM_ESTAT': f'Operativa'})
    stations_meta = pd.DataFrame(res.json())
    return stations_meta

@functools.cache
def get_meteo_variables():
    if os.path.exists('meta_cache/meteo_variables.csv'):
        print('Getting meteo variables from local cache')
        with open('meta_cache/meteo_variables.csv', 'r') as f:
            vars_meta = pd.read_csv(f)
        return vars_meta
            
    print('Fetching meteo station from analisi.transparenciacatalunya.cat...')
    res = requests.get(XEMA_META_VARS)
    vars_meta = pd.DataFrame(res.json())
    return vars_meta
    
def get_meteo_data(date, stations_meta, vars_meta):
    if os.path.exists(f'meta_cache/meteo-meta-{date.isoformat()}.csv'):
        print('Getting meteo variables from local cache')
        with open(f'meta_cache/meteo-meta-{date.isoformat()}.csv', 'r') as f:
            vars_meta = pd.read_csv(f)
        return vars_meta
    res = requests.get(XEMA_DATA_URL, params={'data_lectura': f'{date.isoformat()}T12:00:00.000'})
    xema_day = pd.DataFrame(res.json()).set_index('id')
    
    merged = xema_day.merge(stations_meta, on="codi_estacio").merge(vars_meta, on="codi_variable")
    merged = merged[["codi_estacio", "nom_estacio", "latitud", "longitud", "acronim", "valor_lectura"]]
    
    merged_transposed = (merged.groupby(['codi_estacio', 'nom_estacio', 'latitud', 'longitud'])
                         .apply(lambda group: group[['acronim', 'valor_lectura']].set_index('acronim').T.reset_index(drop=True).rename_axis(None, axis=1))
                         .reset_index().drop(['level_4'], axis=1).set_index('codi_estacio'))
    return merged_transposed

In [38]:
meteo_stations = get_meteo_stations()
meteo_vars = get_meteo_variables()
meteo_data = get_meteo_data(date.value, meteo_stations, meteo_vars)

Getting meteo station from local cache
Getting meteo variables from local cache
Getting meteo variables from local cache


In [39]:
meteo_vars[['acronim', 'nom_variable']]

,acronim,nom_variable
0,PPTx1min,Precipitació màxima en 1 minut
1,HRx,Humitat relativa màxima
2,VV10,Velocitat del vent a 10 m (esc.)
3,DV10,Direcció de vent 10 m (m. 1)
4,T,Temperatura
5,HR,Humitat relativa
6,P,Pressió atmosfèrica
7,PPT,Precipitació
8,RS,Irradiància solar global
9,GNEU,Gruix de neu a terra


In [40]:
meteo_data

,codi_estacio,nom_estacio,latitud,longitud,Tx,Px,Pn,HRx,VV10,DV10,...,GNEU,VV2,DV2,VVx2,DVVx2,RN,VV6,DV6,VVx6,DVVx6
0,C6,Castellnou de Seana,41.65660,0.95172,30.6,984.8,984.4,30.0,5.1,132.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C7,Tàrrega,41.66695,1.16234,29.6,967.3,966.8,24.0,6.2,173.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C8,Cervera,41.67555,1.29609,27.3,953.0,952.8,28.0,6.4,186.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C9,Mas de Barberans,40.71825,0.39988,25.5,989.0,988.8,36.0,4.2,185.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CC,Orís,42.07398,2.20862,28.4,945.0,944.8,29.0,2.9,196.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,VH,Gimenells,41.65814,0.39301,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,VP,Pinós,41.80483,1.53853,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,VX,Tagamanent - PN del Montseny,41.74761,2.30291,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,WQ,Montsec d'Ares (1.572 m),42.05130,0.72952,NaN,NaN,NaN,NaN,5.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Store metadata to local cache

In [41]:
if not os.path.exists('meta_cache/meteo_stations.csv'):
    with open('meta_cache/meteo_stations.csv', 'w') as f:
        meteo_stations.to_csv(f)

if not os.path.exists('meta_cache/meteo_variables.csv'):
    with open('meta_cache/meteo_variables.csv', 'w') as f:
        meteo_vars.to_csv(f)
        
if not os.path.exists(f'meta_cache/meteo-meta-{date.value.isoformat()}.csv'):
    with open(f'meta_cache/meteo-meta-{date.value.isoformat()}.csv', 'w') as f:
        meteo_data.to_csv(f)

Upload metadata to Cloud Object Storage

In [61]:
meteo_key = f'meteo-meta-{date.value.isoformat()}.csv'
try:
    storage.head_object(bucket=DATA_BUCKET, key=meteo_key)
except StorageNoSuchKeyError:
    print('Uploading meteo data to Object Storage...')
    body = meteo_data.to_csv().encode('utf-8')
    storage.put_object(bucket=DATA_BUCKET, key=meteo_key, body=body)

### Digital Terrain Models

Download DTM files for free from http://centrodedescargas.cnig.es/CentroDescargas/buscadorCatalogo.do?codFamilia=MDT05# and put them in `inpit_DTMs` folder.

In [99]:
dtm_asc_keys = storage.list_keys(bucket=DATA_BUCKET, prefix=DTM_ASC_PREFIX)
dtm_asc_keys

['DTMs/asc/PNOA_MDT05_ETRS89_HU31_0359_LID.asc']

Upload tiles from local input folder:

In [103]:
local_dtm_input = 'input_DTMs'
mdts = [os.path.join(mdt_folder, mdt) for mdt in os.listdir(mdt_folder) if mdt.endswith('.asc')]

def upload_file(file_path):
    key = os.path.join(DTM_ASC_PREFIX, os.path.basename(file_path))
    if key in dtm_asc_keys:
        print(f'Tile {key} already in storage')
        return key
    with open(file_path, 'rb') as f:
        print(f'Uploading {key}...')
        storage.put_object(bucket=DATA_BUCKET, key=key, body=f)
    return key

with concurrent.futures.ThreadPoolExecutor(max_workers=16) as pool:
    result = list(pool.map(upload_file, mdts))
    # list(result)

Tile DTMs/asc/PNOA_MDT05_ETRS89_HU31_0359_LID.asc already in storage


In [122]:
def asc_to_geotiff(obj, storage):
    asc_file_name = os.path.basename(obj.key)
    tile_id, _ = os.path.splitext(asc_file_name)
    out_path = os.path.join(tempfile.gettempdir(), tile_id + '.geotiff')
    out_key = os.path.join(DTM_GEOTIFF_PREFIX, tile_id + '.geotiff')
       
    with rasterio.open(obj.data_stream) as src:
        profile = src.profile
        # Cloud optimized GeoTiff parameters
        profile.update(driver='GTiff')
        profile.update(blockxsize=256)
        profile.update(blockysize=256)
        profile.update(tiled=True)
        profile.update(compress='deflate')
        profile.update(interleave='band')
        with rasterio.open(out_path, 'w', **profile) as dest:
            dest.write(src.read())
    
    with open(out_path, 'rb') as f:
        storage.put_object(bucket=DATA_BUCKET, key=out_key, body=f)
    
    return out_key

In [123]:
fexec = lithops.FunctionExecutor()
fs = fexec.map(asc_to_geotiff, os.path.join('localhost://', DATA_BUCKET, DTM_ASC_PREFIX))

2022-06-01 23:59:24,166 [INFO] lithops.config -- Lithops v2.6.0
2022-06-01 23:59:24,171 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-06-01 23:59:24,173 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-06-01 23:59:24,174 [INFO] lithops.invokers -- ExecutorID 83067c-7 | JobID M000 - Selected Runtime: python 
2022-06-01 23:59:24,178 [INFO] lithops.invokers -- ExecutorID 83067c-7 | JobID M000 - Starting function invocation: asc_to_geotiff() - Total: 1 activations
2022-06-01 23:59:24,215 [INFO] lithops.invokers -- ExecutorID 83067c-7 | JobID M000 - View execution logs at /tmp/lithops/logs/83067c-7-M000.log


In [124]:
fexec.get_result(fs=fs)

2022-06-01 23:59:24,310 [INFO] lithops.wait -- ExecutorID 83067c-7 - Getting results from 1 function activations


    0%|          | 0/1  

2022-06-01 23:59:34,351 [INFO] lithops.executors -- ExecutorID 83067c-7 - Cleaning temporary data


['DTMs/geotiff/PNOA_MDT05_ETRS89_HU31_0359_LID.geotiff']

In [125]:
tiles = [os.path.splitext(os.path.basename(mdt))[0] for mdt in mdts]
tiles

['PNOA_MDT05_ETRS89_HU31_0359_LID']

Convert digital elevation map into a Cloud Optimized Geotiff. Upload then to IBM COS:

In [64]:
for mdt in mdts:


In [65]:
mdts_gtiff = [os.path.join(mdt_folder, mdt) for mdt in os.listdir(mdt_folder) if mdt.endswith('.geotiff')]
mdts_gtiff

['input_DTMs/PNOA_MDT05_ETRS89_HU31_0359_LID.tif']

In [ ]:
for mdt_gtiff in mdts_gtiff:
    with open(mdt_gtiff, 'rb') as mdt_file:
        cloud_storage.put_object(bucket=BUCKET, key=mdt_gtiff, body=mdt_file)

## Serverless computation

Input MDT tiles to process:

In [ ]:
tiles = [os.path.splitext(os.path.basename(key))[0]
         for key in cloud_storage.list_keys(bucket=BUCKET, prefix='MDT')]
tiles

In [ ]:
# DEBUG
# tiles = ['PNOA_MDT05_ETRS89_HU30_0933_LID']
# tiles

Compute solar irradiation given a day of year using GRASS libraries:

In [ ]:
def compute_solar_irradiation(inputFile, outputFile, crs='32630'):
    # Define grass working set
    GRASS_GISDB = 'grassdata'
    GRASS_LOCATION = 'GEOPROCESSING'
    GRASS_MAPSET = 'PERMANENT'
    GRASS_ELEVATIONS_FILENAME = 'ELEVATIONS'

    os.environ['GRASSBIN'] = 'grass76'
    from grass_session import Session
    import grass.script as gscript
    from grass.pygrass.modules.shortcuts import general
    from grass.pygrass.modules.shortcuts import raster

    os.environ.update(dict(GRASS_COMPRESS_NULLS='1'))
    
    # Clean previously processed data
    if os.path.isdir(GRASS_GISDB):
        shutil.rmtree(GRASS_GISDB)
    with Session(gisdb=GRASS_GISDB, location=GRASS_LOCATION, mapset=GRASS_MAPSET, create_opts='EPSG:32630') as ses:
        # Set project projection to match elevation raster projection
        general.proj(epsg=crs, flags='c') 
    
        # Load raster file into working directory
        raster.import_(input=inputFile, output=GRASS_ELEVATIONS_FILENAME, flags='o')    
        
        # Set project region to match raster region
        general.region(raster=GRASS_ELEVATIONS_FILENAME, flags='s')    
        # Calculate solar irradiation
        gscript.run_command('r.slope.aspect', elevation=GRASS_ELEVATIONS_FILENAME,
                            slope='slope', aspect='aspect')
        gscript.run_command('r.sun', elevation=GRASS_ELEVATIONS_FILENAME,
                            slope='slope', aspect='aspect', beam_rad='beam',
                            step=1, day=DAY_OF_YEAR)
        
        # Get extraterrestrial irradiation from history metadata
        regex = re.compile(r'\d+\.\d+')
        output = gscript.read_command("r.info", flags="h", map=["beam"])
        splits = str(output).split('\n')
        line = next(filter(lambda line: 'Extraterrestrial' in line, splits))
        extraterrestrial_irradiance = float(regex.search(line)[0])
        
        # Export generated results into a GeoTiff file
        if os.path.isfile(outputFile):
            os.remove(outputFile)
            
        r.out_gdal(input='beam', output=outputFile)
        
        return extraterrestrial_irradiance

Get stations contained in the area of interest:

In [ ]:
def filter_stations(bounds, stations):
    total_points = MultiPoint([Point(x,y) for x, y in stations[['X', 'Y']].to_numpy()])
    intersection = bounds.buffer(AREA_OF_INFLUENCE).intersection(total_points)
    
    return stations[[ intersection.contains(point) for point in total_points]]

Inverse Distance Weighting interpolation:

In [ ]:
def compute_basic_interpolation(shape, stations, field_value, offset = (0,0)):
    station_pixels = [[pixel[0], pixel[1]] for pixel in stations['pixel'].to_numpy()]
    
    # Get an array where each position represents pixel coordinates
    tile_pixels = np.indices(shape).transpose(1,2,0).reshape(shape[0]*shape[1], 2) + offset
    dist = distance_matrix(station_pixels, tile_pixels)
    weights = np.where(dist == 0, np.finfo('float32').max, 1.0 / dist )
    weights /=  weights.sum(axis=0)
    
    return np.dot(weights.T, stations[field_value].to_numpy()).reshape(shape).astype('float32')

Interpolate temperatures from a subset of the tile:

In [ ]:
def radiation_interpolation(obj, block_x, block_y, splits, storage):
    tile_key = os.path.basename(obj.key)
    tile_id, _ = os.path.splitext(tile_key)
    
    with rasterio.open(obj.data_stream) as src:
        transform = src.transform
        
        # Compute working window
        step_w = src.width / splits
        step_h = src.height / splits
        
        offset_h = round(step_h * block_x)
        offset_w = round(step_w * block_y)
        
        profile = src.profile
        width = math.ceil(step_w * (block_y + 1) - offset_w)
        height = math.ceil(step_h * (block_x + 1) - offset_h)
        
        profile.update(width=width)
        profile.update(height=height)
        
        window = Window(offset_w, offset_h, width, height)
        
        with rasterio.open('input', 'w', **profile) as dest:
            dest.write(src.read(window=window))
        
    # Stores global irradiation at "output", it also returns extraterrestrial irradiation
    extraterrestrial_irradiation = compute_solar_irradiation('input', 'output')
        
    # Create and store a raster with extraterrestrial_irradiation
    with rasterio.open('extr', 'w', **profile) as dest:
        data = np.full((height, width), extraterrestrial_irradiation, dtype='float32')
        dest.write(data, 1)
        
    out_key = os.path.join('tmp', 'extrad', tile_id, f'chunk_{block_x}-{block_y}') + '.tif'
    with open('extr', 'rb') as out_file:
        storage.put_object(BUCKET, out_key, out_file)
    
    out_key = os.path.join('tmp', 'rad', tile_id, f'chunk_{block_x}-{block_y}') + '.tif'
    with open('output', 'rb') as out_file:
        storage.put_object(BUCKET, out_key, out_file)
    
    return out_key

In [ ]:
def map_interpolation(obj, block_x, block_y, splits, data_field, storage):    
    tile_key = os.path.basename(obj.key)
    tile_id, _ = os.path.splitext(tile_key)
          
    siam_stream = storage.get_object(BUCKET, 'siam.csv', stream=True)
    siam = pd.read_csv(siam_stream)
    
    with rasterio.open(obj.data_stream) as src:
        transform = src.transform
        
        # Compute working window
        step_w = src.width / splits
        step_h = src.height / splits
        
        offset_h = round(step_h * block_x)
        offset_w = round(step_w * block_y)
        
        profile = src.profile
        
        width = math.ceil(step_w * (block_y + 1) - offset_w)
        height = math.ceil(step_h * (block_x + 1) - offset_h)
        
        profile.update(width=width)
        profile.update(height=height)
        
        window = Window(offset_w,offset_h, width, height)
        
        # Filter desired stations
        bounding_rect = box(src.bounds.left, src.bounds.top, src.bounds.right, src.bounds.bottom)
        filtered = pd.DataFrame(filter_stations(bounding_rect, siam))
        filtered['pixel'] = filtered.apply(
            lambda station: rasterio.transform.rowcol(transform, station['X'], station['Y']), axis=1)
        
        # Interpolate and write results 
        with rasterio.open('output', 'w', **profile) as dest:
            if data_field == 'temp':
                elevations = src.read(1, window=window) # Get elevations content
                interpolation = compute_basic_interpolation(elevations.shape, filtered,
                                                            'tdet', (offset_h, offset_w))
                interpolation += r * (elevations - zdet)
                dest.write(np.where(elevations == src.nodata, np.nan, interpolation), 1)
            else:
                interpolation = compute_basic_interpolation((height, width), 
                                                            filtered, 
                                                            'hr' if data_field == 'humi' else 'v', 
                                                            (offset_h, offset_w))
                dest.write(interpolation, 1)

    # Export results to storage
    out_key = os.path.join('tmp', data_field, tile_id, 'chunk_{}-{}'.format(block_x, block_y)) + '.tif'
    with open('output', 'rb') as output_file:
        storage.put_object(BUCKET, out_key, output_file)
    
    return out_key

Lithops serverless computation:

In [ ]:
iterdata = [('ceph://{}/MDT/{}.tif'.format(BUCKET, tile), i, j) 
            for i in range(SPLITS) for j in range(SPLITS) for tile in tiles]

In [ ]:
pprint(iterdata)
print('Total functions: {} tiles * ({}^2) splits * 4 calculations = {}'.format(
    len(tiles), SPLITS, len(iterdata) * 4))

In [ ]:
fexec = lithops.FunctionExecutor(backend=COMPUTE_BACKEND, storage=STORAGE_BACKEND,
                                 runtime=RUNTIME, log_level="DEBUG")

fexec.map(radiation_interpolation, iterdata, extra_args=(SPLITS,), runtime_memory=2048)

fexec.map(map_interpolation, iterdata, extra_args=(SPLITS,'temp'), runtime_memory=2048)

fexec.map(map_interpolation, iterdata, extra_args=(SPLITS,'humi'), runtime_memory=2048)

fexec.map(map_interpolation, iterdata, extra_args=(SPLITS,'wind'), runtime_memory=2048)

out_chunks = fexec.get_result()

---

### KPIs (Interpolation)

[Skip KPI section](#(End-of-KPI-section---Interpolation))

In [ ]:
fexec.plot(dst="plots/interpolation")

In [ ]:
Image(filename="plots/interpolation_histogram.png")

In [ ]:
Image(filename="plots/interpolation_timeline.png")

#### Data size 

List of .tif files being processed:

In [ ]:
mdts_gtiff

Total size accounting that files were repeatedly processed:

In [ ]:
data_size = sum(obj["Size"] for obj in cloud_storage.list_objects(BUCKET) if obj["Key"] in mdts_gtiff)
data_size *= 4  # Each file was processed 4 times

print(f"Data size: {data_size / 1024**2} MiB")

#### KPI: Cost

In [ ]:
fexec.job_summary()

In [ ]:
pd.read_csv(fexec.log_path)

In [ ]:
cost_interpolation = get_process_cost(fexec)
print(f"The experiment cost ${cost_interpolation:.4f}.")

#### KPI: Throughput

In [ ]:
tstamps = set()
for future in fexec.futures:
    for key in future.stats.keys():
        if key.endswith("tstamp"):
            tstamps.add(future.stats[key])
            
duration = max(tstamps) - min(tstamps)
print("Duration: " + str(duration) + " seconds")

In [ ]:
throughput_interpolation = data_size / duration  # Bytes/second

In [ ]:
print(f"Throughput: {throughput_interpolation / 1024**2} MiB/s")

#### KPI: Speedup

In this section we compare the execution speed of a sample process performed in last section, using different amounts of parallel workers, in order to test the scalability of the process.

In [ ]:
parallel_workers = [12, 24, 48, 72]
experiment_duration = dict.fromkeys(parallel_workers)

Perform experiment several times and save duration:

In [ ]:
for option in parallel_workers:
    fexec = lithops.FunctionExecutor(
        backend=COMPUTE_BACKEND, 
        storage=STORAGE_BACKEND,
        runtime=RUNTIME,
        workers=option, # Tells lithops to work w/only this number of concurrent workers
        log_level="DEBUG"
    )
    fexec.map(
        map_interpolation, iterdata, extra_args=(SPLITS,'temp'), runtime_memory=2048
    )
    fexec.get_result()
    
    tstamps = set()
    for future in fexec.futures:
        for key in future.stats.keys():
            if key.endswith("tstamp"):
                tstamps.add(future.stats[key])
    duration = max(tstamps) - min(tstamps)
    experiment_duration[option] = duration

In [ ]:
experiment_duration

##### Visualization of per-worker performance relative to first experiment

The following plot represents two lines:
- **Ideal speedup**: theoretical best speedup - scenario where a 2x increment in workers results in 1/2 execution time, a 4x increment in workers results in a 1/4 execution time, etc.
- **Lithops speedup**: actual speedup that results from the experiment

In [ ]:
duration = list(experiment_duration.values())
theoretical_best_speedup = [(1 - parallel_workers[0] / parallel_workers[i]) * 100 for i in range(0, len(parallel_workers))]
actual_speedup = [(1 - duration[i] / duration[0]) * 100 for i in range(0, len(duration))]

plt.plot(
    parallel_workers,
    theoretical_best_speedup
)
plt.plot(
    parallel_workers,
    actual_speedup
)
plt.xlabel("Number of workers")
plt.ylabel("% time reduced, relative to first experiment")
plt.legend(["Ideal speedup", "Lithops speedup (this experiment)"])

Clean stats:

In [ ]:
fexec.futures = []

###### (End of KPI section - Interpolation)

---

Join split subsets into a tile:

In [ ]:
def gather_blocks(tile, splits, data_field, storage):

    from rasterio.windows import Window
    
    # Get width and height from original tile
    with rasterio.open(storage.get_object(bucket=BUCKET, key=f'MDT/{tile}.tif', stream=True)) as og:
        height = og.profile['height']
        width = og.profile['width']
    
    chunk_tiles = storage.list_keys(bucket=BUCKET, prefix=f'tmp/{data_field}/{tile}/chunk')
        
    # Open first object to obtain profile metadata
    with rasterio.open(storage.get_object(bucket=BUCKET, key=chunk_tiles[0], stream=True)) as src:
        profile = src.profile
        profile.update(width=width)
        profile.update(height=height)

    # Iterate each object and print its block into the destination file
    with rasterio.open("output", "w", **profile) as dest: 
        for chunk in chunk_tiles:
            j, i = os.path.splitext(os.path.basename(chunk))[0].rsplit('_')[1].split('-')
            j, i = int(j), int(i)
            with rasterio.open(storage.get_object(bucket=BUCKET, key=chunk, stream=True)) as src:
                step_w = math.floor(width / splits)
                step_h = math.floor(height / splits)
                curr_window = Window(round(step_w * i), round(step_h * j), src.width, src.height)
                content = src.read(1)
                dest.write(content, 1, window=curr_window)
            # storage.delete_object(bucket=BUCKET, key=chunk)
    
    output_key = os.path.join('tmp', data_field, tile, '_'.join([tile, data_field.upper()+'.tif']))
    with open('output', 'rb') as out_file:
        storage.put_object(bucket=BUCKET, key=output_key, body=out_file)  
    
    return output_key

Combine previous split subsets:

In [ ]:
fexec.map(gather_blocks, tiles, extra_args=(SPLITS, 'extrad'), runtime_memory=2048)

fexec.map(gather_blocks, tiles, extra_args=(SPLITS, 'humi'), runtime_memory=2048)

fexec.map(gather_blocks, tiles, extra_args=(SPLITS, 'rad'), runtime_memory=2048)

fexec.map(gather_blocks, tiles, extra_args=(SPLITS, 'temp'), runtime_memory=2048)

fexec.map(gather_blocks, tiles, extra_args=(SPLITS, 'wind'), runtime_memory=2048)

out_combined = fexec.get_result()

### KPIs (Gather blocks)

[Skip KPI section](#(End-of-KPI-section---Gather-blocks))

In [ ]:
fexec.plot(dst="plots/gather_blocks")

In [ ]:
Image(filename="plots/gather_blocks_histogram.png")

In [ ]:
Image(filename="plots/gather_blocks_timeline.png")

#### Data size

In [ ]:
mdts_gtiff

In [ ]:
data_size = sum(obj["Size"] for obj in cloud_storage.list_objects(BUCKET) if obj["Key"] in mdts_gtiff)
data_size *= 4  # Each file was processed 4 times

print(f"Data size: {data_size / 1024**2} MiB")

#### KPI: Cost

In [ ]:
fexec.job_summary()

In [ ]:
pd.read_csv(fexec.log_path)

In [ ]:
cost_gather_blocks = get_process_cost(fexec)
print(f"The experiment cost ${cost_gather_blocks:.4f}.")

#### KPI: Throughput

In [ ]:
tstamps = set()
for future in fexec.futures:
    for key in future.stats.keys():
        if key.endswith("tstamp"):
            tstamps.add(future.stats[key])
            
duration = max(tstamps) - min(tstamps)
print("Duration: " + str(duration) + " seconds")

In [ ]:
throughput_gather_blocks = data_size / duration  # Bytes/second

In [ ]:
print(f"Throughput: {throughput_gather_blocks / 1024**2} MiB/s")

Clean stats:

In [ ]:
fexec.futures = []

###### (End of KPI section - Gather blocks)

---

## Computation of potential evaporation

In [ ]:
def compute_crop_evapotranspiration(temperatures,
                                    humidities,
                                    wind_speeds,
                                    external_radiations,
                                    global_radiations,
                                    KCs):
    gamma = 0.665*101.3/1000
    eSat = 0.6108 * np.exp((17.27*temperatures)/(temperatures+237.3))
    delta = 4098 * eSat / np.power((temperatures + 237.3),2)
    eA = np.where(humidities < 0, 0, eSat * humidities / 100)     # Avoid sqrt of a negative number
    T4 = 4.903 * np.power((273.3 + temperatures),4)/1000000000
    rSrS0 = global_radiations/(external_radiations * 0.75)
    rN = 0.8* global_radiations-T4*(0.34-0.14*np.sqrt(eA))*((1.35*rSrS0)-0.35)
    den = delta + gamma *(1 + 0.34* wind_speeds)
    tRad = 0.408 * delta * rN / den
    tAdv = gamma * (900/(temperatures+273))*wind_speeds * (eSat - eA)/den
    return ((tRad + tAdv) * 7 * KCs).astype('float32')

In [ ]:
vineyard = ['VI', 'VO', 'VF', 'FV', 'CV' ]
olive_grove = ['OV', 'VO', 'OF', 'FL', 'OC']
fruit = ['FY', 'VF', 'OF', 'FF', 'CF']
nuts = ['FS', 'FV', 'FL', 'FF', 'CS' ]
citrus = ['CI', 'CV', 'OC', 'CF', 'CS' ]

def get_kc(feature):
    
    # TODO: Get more precise values of Kc
    sigpac_use = feature['properties']['uso_sigpac']
    if sigpac_use in vineyard:
        # Grapes for wine - 0.3, 0.7, 0.45
        return 0.7  
    if sigpac_use in olive_grove:
        # Olive grove - ini: 0.65, med: 0.7, end: 0.7
        return 0.7 
    if sigpac_use in fruit:
        # Apples, Cherries, Pears - 0.45, 0.95, 0.7
        return 0.95
    if sigpac_use in nuts:
        # Almonds - 0.4, 0.9, 0.65
        return 0.9
    if sigpac_use in citrus:
        # Citrus, without ground coverage - 0.7, 0.65, 0.7
        return 0.65
    
    return None

In [ ]:
def get_geometry_window(src, geom_bounds):
    left, bottom, right, top = geom_bounds
    src_left, src_bottom, src_right, src_top = src.bounds
    window = src.window(max(left,src_left), max(bottom,src_bottom), min(right,src_right), min(top,src_top))
    window_floored = window.round_offsets(op='floor', pixel_precision=3)
    w = math.ceil(window.width + window.col_off - window_floored.col_off)
    h = math.ceil(window.height + window.row_off - window_floored.row_off)
    return Window(window_floored.col_off, window_floored.row_off, w, h)     

In [ ]:
def compute_evapotranspiration_by_shape(tem, hum, win, rad, extrad, dst):
    
    import fiona
    from shapely.geometry import shape, box
    from rasterio import features
    
    non_arable_land = ['AG', 'CA', 'ED', 'FO', 'IM', 'PA', 'PR', 'ZU', 'ZV']
    
    with fiona.open('zip://shape.zip') as shape_src:
        for feature in shape_src.filter(bbox=tem.bounds):
            KC = get_kc(feature) 
            if KC is not None:   
                geom = shape(feature['geometry'])  
                window = get_geometry_window(tem, geom.bounds)              
                win_transform = rasterio.windows.transform(window, tem.transform)
                # Convert shape to raster matrix
                image = features.rasterize([geom],
                                           out_shape=(window.height, window.width),
                                           transform = win_transform,
                                           fill = 0,
                                           default_value = 1).astype('bool')
                # Get values to compute evapotranspiration
                temperatures = tem.read(1, window=window)
                humidities = hum.read(1, window=window)
                wind_speeds = win.read(1, window=window)
                # Convert from W to MJ (0.0036)
                global_radiations = rad.read(1, window=window) * 0.0036
                external_radiations = extrad.read(1, window=window) * 0.0036
                KCs = np.full(temperatures.shape, KC)
                # TODO: compute external radiation
                #external_radiations = np.full(temperatures.shape, 14)
                # TODO: compute global radiation
                # global_radiations = np.full(temperatures.shape, 10)
                etc = compute_crop_evapotranspiration(
                        temperatures,
                        humidities,
                        wind_speeds,
                        external_radiations,
                        global_radiations,
                        KCs
                )
                etc[temperatures == tem.nodata] = dst.nodata
                etc[np.logical_not(image)] = dst.nodata
                dst.write(etc + dst.read(1, window=window), 1, window=window)

In [ ]:
def compute_global_evapotranspiration(tem, hum, win, rad, extrad, dst):    
    for ji, window in tem.block_windows(1):
        bounds = rasterio.windows.bounds(window, tem.transform)
        temperatures = tem.read(1, window=window)
        humidities = hum.read(1, window=window)
        wind_speeds = win.read(1, window=window)
         # Convert from W to MJ (0.0036)
        global_radiations = rad.read(1, window=window) * 0.0036
        external_radiations = extrad.read(1, window=window) * 0.0036
        # TODO: compute external radiation
        #external_radiations = np.full(temperatures.shape, 14)
        # TODO: compute global radiation
        # global_radiations = np.full(temperatures.shape, 10)
        # TODO: compute KCs
        KCs = np.full(temperatures.shape, 1)
        etc = compute_crop_evapotranspiration(
                temperatures,
                humidities,
                wind_speeds,
                external_radiations,
                global_radiations,
                KCs
        )
        dst.write(np.where(temperatures == tem.nodata, dst.nodata, etc), 1, window=window)

In [ ]:
def combine_calculations(tile, storage):
    
    from functools import partial
      
    # Download shapefile
    shapefile = storage.get_object(bucket=BUCKET, key='shapefile.zip', stream=True)
    with open('shape.zip', 'wb') as shapf:
        for chunk in iter(partial(shapefile.read, 200 * 1024 * 1024), ''):
            if not chunk:
                break
            shapf.write(chunk)
    
    temp = storage.get_object(bucket=BUCKET, key=f'tmp/temp/{tile}/{tile}_TEMP.tif', stream=True)
    humi = storage.get_object(bucket=BUCKET, key=f'tmp/humi/{tile}/{tile}_HUMI.tif', stream=True)
    rad = storage.get_object(bucket=BUCKET, key=f'tmp/rad/{tile}/{tile}_RAD.tif', stream=True)
    extrad = storage.get_object(bucket=BUCKET, key=f'tmp/extrad/{tile}/{tile}_EXTRAD.tif', stream=True)
    wind = storage.get_object(bucket=BUCKET, key=f'tmp/wind/{tile}/{tile}_WIND.tif', stream=True)
    
    with rasterio.open(temp) as temp_raster:
        with rasterio.open(humi) as humi_raster:
            with rasterio.open(rad) as rad_raster:
                with rasterio.open(extrad) as extrad_raster:
                    with rasterio.open(wind) as wind_raster:
                        profile = temp_raster.profile
                        profile.update(nodata=0)
        
                        with rasterio.open('output', 'w+', **profile) as dst:
#                             compute_global_evapotranspiration(temp_raster, humi_raster, wind_raster,
#                                                               rad_raster, extrad_raster, dst)
                            compute_evapotranspiration_by_shape(temp_raster, humi_raster, wind_raster,
                                                                rad_raster, extrad_raster, dst)
    
    out_key = f'etc/{tile}_ETC.tif'
    with open('output', 'rb') as output_f:
        storage.put_object(bucket=BUCKET, key=out_key, body=output_f)
    return out_key

In [ ]:
fexec.map(combine_calculations, tiles, runtime_memory=2048)

res = fexec.get_result()

### KPIs (Potential evaporation)

[Skip KPI section](#(End-of-KPI-section---Potential-evaporation))

In [ ]:
fexec.plot(dst="plots/potential_evaporation")

In [ ]:
Image(filename="plots/potential_evaporation_histogram.png")

In [ ]:
Image(filename="plots/potential_evaporation_timeline.png")

#### Data size

In [ ]:
tiles

In [ ]:
data_size = 0

for obj in cloud_storage.list_objects(BUCKET):
    for tile in tiles:
        if obj["Key"] == f'tmp/temp/{tile}/{tile}_TEMP.tif' or \
                obj["Key"] == f'tmp/humi/{tile}/{tile}_TEMP.tif' or \
                obj["Key"] == f'tmp/rad/{tile}/{tile}_TEMP.tif' or \
                obj["Key"] == f'tmp/extrad/{tile}/{tile}_TEMP.tif' or \
                obj["Key"] == f'tmp/wind/{tile}/{tile}_TEMP.tif' or \
                obj["Key"] == 'shapefile.zip':
            data_size += obj["Size"]

print(f"Data size: {data_size / 1024**2} MiB")

#### KPI: Cost

In [ ]:
fexec.job_summary()

In [ ]:
pd.read_csv(fexec.log_path)

In [ ]:
cost_potential_evaporation = get_process_cost(fexec)
print(f"The experiment cost ${cost_potential_evaporation:.4f}.")

#### KPI: Throughput

In [ ]:
tstamps = set()
for future in fexec.futures:
    for key in future.stats.keys():
        if key.endswith("tstamp"):
            tstamps.add(future.stats[key])
            
duration = max(tstamps) - min(tstamps)
print("Duration: " + str(duration) + " seconds")

In [ ]:
throughput_potential_evaporation = data_size / duration  # Bytes/second

In [ ]:
print(f"Throughput: {throughput_potential_evaporation / 1024**2} MiB/s")

#### KPI: Speedup

In this section we compare the execution speed of a sample process performed in last section, using different amounts of parallel workers, in order to test the scalability of the process.

In [ ]:
parallel_workers = [2, 4, 8]
experiment_duration = dict.fromkeys(parallel_workers)

Perform experiment several times and save duration:

In [ ]:
for option in parallel_workers:
    fexec = lithops.FunctionExecutor(
        backend=COMPUTE_BACKEND, 
        storage=STORAGE_BACKEND, 
        runtime=RUNTIME,
        workers=option, # Tells lithops to work w/only this number of concurrent workers
        log_level="NOTSET"
    )
    fexec.map(combine_calculations, tiles, runtime_memory=2048)
    fexec.get_result()
    
    tstamps = set()
    for future in fexec.futures:
        for key in future.stats.keys():
            if key.endswith("tstamp"):
                tstamps.add(future.stats[key])
    duration = max(tstamps) - min(tstamps)
    experiment_duration[option] = duration

In [ ]:
experiment_duration

##### Visualization of per-worker performance relative to first experiment

The following plot represents two lines:
- **Ideal speedup**: theoretical best speedup - scenario where a 2x increment in workers results in 1/2 execution time, a 4x increment in workers results in a 1/4 execution time, etc.
- **Lithops speedup**: actual speedup that results from the experiment

In [ ]:
duration = list(experiment_duration.values())
theoretical_best_speedup = [(1 - parallel_workers[0] / parallel_workers[i]) * 100 for i in range(0, len(parallel_workers))]
actual_speedup = [(1 - duration[i] / duration[0]) * 100 for i in range(0, len(duration))]

plt.plot(
    parallel_workers,
    theoretical_best_speedup
)
plt.plot(
    parallel_workers,
    actual_speedup
)
plt.xlabel("Number of workers")
plt.ylabel("% time reduced, relative to first experiment")
plt.legend(["Ideal speedup", "Lithops speedup (this experiment)"])

Clean stats:

In [ ]:
fexec.futures = []

###### (End of KPI section - Potential evaporation)

---

## Visualization of results

In [ ]:
import io
tile = random.choice(tiles)
obj = io.BytesIO(cloud_storage.get_object(bucket=BUCKET, key=f'etc/{tile}_ETC.tif'))

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots()

with rasterio.open(obj) as src:
    arr = src.read(1, out_shape=(src.height, src.width))
    ax.set_title(tile)
    img = ax.imshow(arr, cmap='Greens')
    fig.colorbar(img, shrink=0.5)

fig.set_size_inches(18.5, 10.5)
plt.show()

obj.seek(0)

### Utility functions

Remove intermediate data

In [ ]:
# keys = cloud_storage.list_keys(bucket=BUCKET, prefix='')
# keys

In [ ]:
# for key in keys:
#     cloud_storage.delete_object(bucket=BUCKET, key=key)